<H1> Introdução </H1>
<p> 
    Os dados analisados aqui estão disponíveis no site do Instituto Nacional de Metereologia (INMET) em formato CSV e podem ser baixados através <a href="https://portal.inmet.gov.br/dadoshistoricos"> deste link </a> e por isso não foram incluídos no repositório. 
</p>

<p> 
    Os dados foram mantidos na organização original, com as tabelas divididas em pastas por ano e em arquivos segundo o local de medição. 
</p>

<p> Uma lista de todos os arquivos disponíveis foi salva no arquivo <i> files.txt </i> para facilitar a consulta dos mesmos e para permitir a quem esteja acessando este repositório consultar rapidamente quais arquivos estão disponíveis sem necessidade de baixá-los no site do INMET. </p>

<p> 
Os dados de 2024 não foram incluídos pois no momento em que este repositório foi criado o ano ainda não tinha acabado. Estes dados serão incluídos em algum momento no futuro. 
</p>

In [125]:
# Bibliotecas utilizadas
from datetime import datetime
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h1> Sobre os dados</h1>
<p> Cada arquivo CSV contém dados recolhidos a cada hora em um único local de medição durante um período máximo de um ano.</p>
<p> O período observado em cada arquivo sempre segue o ano civil. Deste modo, não há dados de dois anos diferentes em um mesmo arquivo. </p>
<p> O nome dos arquivos é padronizado no formato <i>INMET_SiglaRegiao_SiglaEstado_Codigo_Local_DataInicial_A_DataFinal.csv </i> (em letras maiúsculas), que é fácil de ser lido e que descreve os metadados dos mesmos, possibilitando uma rápida recuperação das informações desejadas. </p>
<p> <b>Observação:</b> Algumas poucas cidades possuem mais de um local de medição, mas por simplicidade será utilizado apenas o termo 'Cidade' para se referir aos locais nas tabelas e gráficos a seguir. </p>

In [26]:
# Lê a lista de arquivos e criar um dataframe com os dados. 
file = open("files.txt", "r")
file_content = file.read()
file_list = [x for x in file_content.splitlines() if len(x) > 2]
file_list_splitted = [ x.replace(".CSV","").strip().split('_')[0 : 8] + [x] for x in file_list]
df_files = pd.DataFrame(columns=["INMET", "REGIAO", "ESTADO", "CODIGO", "CIDADE", "DATA INI", "A", "DATA FIM", "ARQUIVO"],
                        data = file_list_splitted).drop(columns={"INMET", "A"})
# Mostra a tabela co
display(df_files)

,REGIAO,ESTADO,CODIGO,CIDADE,DATA INI,DATA FIM,ARQUIVO
0,CO,DF,A001,BRASILIA,07-05-2000,31-12-2000,INMET_CO_DF_A001_BRASILIA_07-05-2000_A_31-12-2...
1,NE,BA,A401,SALVADOR,13-05-2000,31-12-2000,INMET_NE_BA_A401_SALVADOR_13-05-2000_A_31-12-2...
2,N,AM,A101,MANAUS,09-05-2000,31-12-2000,INMET_N_AM_A101_MANAUS_09-05-2000_A_31-12-2000...
3,SE,RJ,A601,ECOLOGIA AGRICOLA,07-05-2000,31-12-2000,INMET_SE_RJ_A601_ECOLOGIA AGRICOLA_07-05-2000_...
4,S,RS,A801,PORTO ALEGRE,22-09-2000,31-12-2000,INMET_S_RS_A801_PORTO ALEGRE_22-09-2000_A_31-1...
...,...,...,...,...,...,...,...
8889,S,SC,A867,ARARANGUA,01-01-2023,31-12-2023,INMET_S_SC_A867_ARARANGUA_01-01-2023_A_31-12-2...
8890,S,SC,A868,ITAJAI,01-01-2023,31-12-2023,INMET_S_SC_A868_ITAJAI_01-01-2023_A_31-12-2023...
8891,S,SC,A870,RANCHO QUEIMADO,01-01-2023,31-12-2023,INMET_S_SC_A870_RANCHO QUEIMADO_01-01-2023_A_3...
8892,S,SC,A895,CHAPECO,01-01-2023,31-12-2023,INMET_S_SC_A895_CHAPECO_01-01-2023_A_31-12-202...


<h2>Dados sobre os locais de medição</h2>

In [150]:
print(f"Existem dados de {len(df_files['CIDADE'].unique())} locais diferentes")

Existem dados de 632 locais diferentes


In [156]:
composed_places = [y[0] for y in [x.split(' - ') for x in df_files[df_files['CIDADE'].str.contains(' - ')]['CIDADE'].unique()]] +\
                  [y[0] for y in [x.split(' (') for x in df_files[df_files['CIDADE'].str.contains('\(')]['CIDADE'].unique()]]

duplicated_cities = pd.concat([pd.Series(df_files[df_files['CIDADE'].isin(composed_places)]['CIDADE'].unique()),\
                               pd.Series([item for item, count in Counter(composed_places).items() if count > 1])]).to_list()
print("Cidades com mais de um local de medição: " + ', '.join(duplicated_cities))

Cidades com mais de um local de medição: SALVADOR, PORTO ALEGRE, CRISTALINA, NOVA FRIBURGO, SAQUAREMA, CAMPOS DOS GOYTACAZES, SAO PAULO, BELO HORIZONTE, RIO DE JANEIRO, BRASNORTE


In [28]:
# FUNCAO PARA DEFINIR AS CORES DA PRIMEIRA COLUNA EM TABELAS COM AGRUPAMENTO + REORDANAÇÂO DE DADOS
def color(x):
    no_repetitions = x.unique()
    return ["background-color: #f5f5f5; background: #f5f5f5" if np.where(no_repetitions == el)[0]%2 == 0 \
            else "background-color: #ffffff; background: #ffffff" for  index, el in x.items()]

In [117]:
# EXIBIÇÃO RESUMIDA DAS LOCALIDADES MONITORADAS POR REGIÃO E ESTADO. 
display(df_files[['REGIAO', 'ESTADO', 'CIDADE']]\
            .groupby(by=['REGIAO','ESTADO'])\
            .agg(['nunique', lambda col: ',  '.join([x.title() for x in col.sort_values().unique()])])\
            .sort_values(by=['REGIAO', ('CIDADE','<lambda_0>')], ascending=[True,False])\
            .rename(columns={ 'nunique': 'NÚMERO DE CIDADES', '<lambda_0>': 'CIDADES'}, level=1)\
            .droplevel(0, axis=1)\
            .style.apply_index(color, axis=0, level =[0]))
